In [1]:
import os
import numpy as np
from infer import Inferer

In [2]:
gpus = "3"
n_gpus = len(gpus.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = gpus
inferer = Inferer()
#inferer.run(n_gpus)

['/home1/gzy/NucleiSegmentation/High_CCRCC/Train/512x512_256x256_dist/']
['/home1/gzy/NucleiSegmentation/High_CCRCC/Valid/512x512_256x256_dist/']


Evaluate the predict results and Generate predict overlay images

In [3]:
import cv2
import glob
import itertools
from scipy import io as sio
import matplotlib.pyplot as plt
from tqdm import tqdm

from misc.viz_utils import visualize_instances
from postproc.hover import proc_np_hv
from postproc.other import process
from postproc.dist import process as dist_process
from postprocessing import *
from scipy.ndimage import filters, measurements

def load_ground_trues(ann_dir, basename):
    ann_inst = np.loadtxt(ann_dir + '/' + basename.replace('original','boundary') + '.txt', delimiter=',')
    ann_type_list = list(np.loadtxt(ann_dir + '/' + basename.replace('original','boundary_label') + '.txt', delimiter=','))
    ann_type = ann_inst.copy()
    for i in range(len(ann_type_list)-1):
        ann_type[ann_type==(i+1)] = ann_type_list[i+1]
    ann_type[ann_type == 5] = 4
    ann_type[ann_type == -1] = 0
    ann_inst[ann_inst == -1] = 0
    return ann_inst, ann_type

def load_ground_trues_consep(ann_dir, basename):
    ann = sio.loadmat(ann_dir + '/' + basename + '.mat')
    ann_type = ann['type_map']
    ann_inst = ann['inst_map']
    ann_type[(ann_type == 3) | (ann_type == 4)] = 3
    ann_type[(ann_type == 5) | (ann_type == 6) | (ann_type == 7)] = 4
    return ann_inst, ann_type

def load_ground_trues_ccrcc(ann_dir, basename):
    labels = sio.loadmat(ann_dir + '/' + basename + '.mat')
    ann_inst = labels['instance_map']
    ann_type = labels['class_map']
#     ann_type[ann_type!=0] = 1
    return ann_inst, ann_type

def load_ground_trues_monuseg(ann_dir, basename):
    ann = np.load(ann_dir + '/' + basename + '.npy')
    ann_type = ann.copy()
    ann_type[ann_type!=0] = 1
    ann_inst = ann
    return ann_inst, ann_type

def class_colour(class_value):
    """
    Generate RGB colour for overlay based on class id
    Args:
        class_value: integer denoting the class of object  
    """
    if class_value == 0:
        return 0, 0, 0  # black (background)
    if class_value == 1:
        return 0, 255, 0  # green
    elif class_value == 2:
        return 255, 255, 0  # yellow
    elif class_value == 3:
        return 255, 0, 0  # red
    elif class_value == 4:
        return 0, 0, 255  # blue
    elif class_value == 5:
        return 255, 165, 0  # orange
    elif class_value == 6:
        return 0, 255, 255  # cyan
    else:
        raise Exception(
            'Currently, overlay_segmentation_results() only supports up to 6 classes.')
####

def process_instance(pred_map, nr_types=0, remap_label=False, output_dtype='uint16'):
    """
    Post processing script for image tiles

    Args:
        pred_map: commbined output of nc, np and hv branches
        nr_types: number of types considered at output of nc branch
        remap_label: whether to map instance labels from 1 to N (N = number of nuclei)
        output_dtype: data type of output
    
    Returns:
        pred_inst:     pixel-wise nuclear instance segmentation prediction
        pred_type_out: pixel-wise nuclear type prediction 
    """

    pred_inst = pred_map[..., nr_types:]
    pred_inst = np.squeeze(pred_inst)
    
    pred_inst = proc_np_hv(pred_inst)

    # remap label is very slow - only uncomment if necessary to map labels in order
#     if remap_label:
#         pred_inst = remap_label(pred_inst, by_size=True)
    if nr_types != 0:
        pred_type = pred_map[..., :nr_types]
        pred_type = np.argmax(pred_type, axis=-1)
        pred_type = np.squeeze(pred_type)

        pred_type_out = np.zeros([pred_type.shape[0], pred_type.shape[1]])               
        #### * Get class of each instance id, stored at index id-1
        pred_id_list = list(np.unique(pred_inst))[1:] # exclude background ID
        pred_inst_type = np.full(len(pred_id_list), 0, dtype=np.int32)
        for idx, inst_id in enumerate(pred_id_list):
            inst_tmp = pred_inst == inst_id
            inst_type = pred_type[pred_inst == inst_id]
            type_list, type_pixels = np.unique(inst_type, return_counts=True)
            type_list = list(zip(type_list, type_pixels))
            type_list = sorted(type_list, key=lambda x: x[1], reverse=True)
            inst_type = type_list[0][0]
            if inst_type == 0: # ! pick the 2nd most dominant if exist
                if len(type_list) > 1:
                    inst_type = type_list[1][0]
            pred_type_out += (inst_tmp * inst_type)
    else:
        pred_type_out = pred_inst.copy()
        pred_type_out[pred_type_out!=0] = 1
    
    return pred_inst.astype(output_dtype), pred_type_out.astype(output_dtype)

def process_instance_micro(pred_map, pred_inst=None, nr_types=0, output_dtype='uint16'):
    pred_map = pred_map[0]
    if pred_inst is None:
        pred_inst = pred_map[...,-1].copy()
#         pred_inst = 1 - pred_inst
        pred_inst = process(pred_inst, model_mode='micronet')
    
    if nr_types != 0:
        pred_type = pred_map[..., 1:nr_types]
        pred_type = np.argmax(pred_type, axis=-1)
        pred_type = pred_type + 1
        pred_type = np.squeeze(pred_type)

        pred_type_out = np.zeros([pred_type.shape[0], pred_type.shape[1]])               
        #### * Get class of each instance id, stored at index id-1
        pred_id_list = list(np.unique(pred_inst))[1:] # exclude background ID
        pred_inst_type = np.full(len(pred_id_list), 0, dtype=np.int32)
        for idx, inst_id in enumerate(pred_id_list):
            inst_tmp = pred_inst == inst_id
            inst_type = pred_type[pred_inst == inst_id]
            type_list, type_pixels = np.unique(inst_type, return_counts=True)
            type_list = list(zip(type_list, type_pixels))
            type_list = sorted(type_list, key=lambda x: x[1], reverse=True)
            inst_type = type_list[0][0]
            if inst_type == 0: # ! pick the 2nd most dominant if exist
                if len(type_list) > 1:
                    inst_type = type_list[1][0]
            pred_type_out += (inst_tmp * inst_type)
    else:
        pred_type_out = pred_inst.copy()
        pred_type_out[pred_type_out!=0] = 1

    return pred_inst.astype(output_dtype), pred_type_out.astype(output_dtype)

def process_instance_hcnet(pred_map, nr_types=0, output_dtype='uint16'):
    pred_map = pred_map[0]
    pos_map = pred_map[...,-1]*255
    binary_map = pred_map[...,-2]*255
    pred_inst = gaussianmap2binary(pos_map.astype('uint8'), binary_map.astype('uint8'), 150, 40)
    
    if nr_types != 0:
        pred_type = pred_map[..., 1:nr_types]
        pred_type = np.argmax(pred_type, axis=-1)
        pred_type = pred_type + 1
        pred_type = np.squeeze(pred_type)

        pred_type_out = np.zeros([pred_type.shape[0], pred_type.shape[1]])               
        #### * Get class of each instance id, stored at index id-1
        pred_id_list = list(np.unique(pred_inst))[1:] # exclude background ID
        pred_inst_type = np.full(len(pred_id_list), 0, dtype=np.int32)
        for idx, inst_id in enumerate(pred_id_list):
            inst_tmp = pred_inst == inst_id
            inst_type = pred_type[pred_inst == inst_id]
            type_list, type_pixels = np.unique(inst_type, return_counts=True)
            type_list = list(zip(type_list, type_pixels))
            type_list = sorted(type_list, key=lambda x: x[1], reverse=True)
            inst_type = type_list[0][0]
            if inst_type == 0: # ! pick the 2nd most dominant if exist
                if len(type_list) > 1:
                    inst_type = type_list[1][0]
            pred_type_out += (inst_tmp * inst_type)
    else:
        pred_type_out = pred_inst.copy()
        pred_type_out[pred_type_out!=0] = 1

    return pred_inst.astype(output_dtype), pred_type_out.astype(output_dtype)

def process_instance_mrcnn(pred_map, output_dtype='uint16'):
    masks = pred_map['instance_map']
    classes = pred_map['label'][0]
    pred_inst = masks
    pred_type_out = np.zeros([masks.shape[0], masks.shape[1]])
    for i in range(len(np.unique(masks))-1):
        pred_type_out[masks == (i+1)] = classes[i]
    return pred_inst.astype(output_dtype), pred_type_out.astype(output_dtype)

def bounding_box(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    # due to python indexing, need to add 1 to max
    # else accessing will be 1px in the box, not out 
    rmax += 1
    cmax += 1
    return [rmin, rmax, cmin, cmax]

def visualize_instances(input_image, predict_instance, predict_type=None, line_thickness=2):
    """
    Overlays segmentation results on image as contours
    Args:
        input_image: input image
        predict_instance: instance mask with unique value for every object
        predict_type: type mask with unique value for every class
        line_thickness: line thickness of contours
    Returns:
        overlay: output image with segmentation overlay as contours
    """
   
    overlay = np.copy((input_image).astype(np.uint8))

    if predict_type is not None:
        type_list = list(np.unique(predict_type))  # get list of types
        type_list.remove(0)  # remove background
    else:
        type_list = [0]  # yellow

    for iter_type in type_list:
        if predict_type is not None:
            label_map = (predict_type == iter_type) * predict_instance
        else:
            label_map = predict_instance
        instances_list = list(np.unique(label_map))  # get list of instances
        instances_list.remove(0)  # remove background
        contours = []
        for inst_id in instances_list:
            instance_map = np.array(
                predict_instance == inst_id, np.uint8)  # get single object
            y1, y2, x1, x2 = bounding_box(instance_map)
            y1 = y1 - 2 if y1 - 2 >= 0 else y1
            x1 = x1 - 2 if x1 - 2 >= 0 else x1
            x2 = x2 + 2 if x2 + 2 <= predict_instance.shape[1] - 1 else x2
            y2 = y2 + 2 if y2 + 2 <= predict_instance.shape[0] - 1 else y2
            inst_map_crop = instance_map[y1:y2, x1:x2]
            contours_crop = cv2.findContours(
                inst_map_crop, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            index_correction = np.asarray([[[[x1, y1]]]])
            for i in range(len(contours_crop[0])):
                contours.append(
                    list(np.asarray(contours_crop[0][i].astype('int32')) + index_correction))
        contours = list(itertools.chain(*contours))
        cv2.drawContours(overlay, np.asarray(contours), -1,
                         class_colour(iter_type), line_thickness)
    return overlay

from pq_metrics import get_fast_pq, remap_label, binarize

def cal_metrics(ann_inst, ann_type, pred_inst, pred_type):
    bound_true = remap_label(ann_inst)
    bound_pred = remap_label(pred_inst)
    [dq, sq, pq_bin], _ = get_fast_pq(bound_true, bound_pred)

    cls_true = ann_type
    cls_pred = pred_type
    pq = []
    for c in range(1, inferer.nr_types):
        temp_pred = bound_pred.copy()
        temp_pred[cls_pred!=c] = 0
        temp_pred = remap_label(temp_pred) 
        temp_true = bound_true.copy()
        temp_true[cls_true!=c] = 0
        temp_true = remap_label(temp_true)
        if len(np.unique(temp_true)) == 1:
            tmp_pq = np.nan
        else:
            [_, _, tmp_pq], _ = get_fast_pq(temp_true, temp_pred)
        pq.append(tmp_pq)
        # count tp fp
        total[c-1] += len(np.unique(temp_true)-1)
        predict[c-1] += len(np.unique(temp_pred)-1)
        for j in np.unique(temp_true):
            if j == 0:
                continue
            # 重叠>0.5 count
            if np.count_nonzero(temp_pred[temp_true == j]) > temp_pred[temp_true == j].size*.5:
                correct[c-1] += 1
    return dq, sq, pq_bin, pq

# pred_path = "./output/High_CCRCC/unet"
pred_path = "/home1/sjb/project_source_code/Nuclei-Segmentation-using-Mask-RCNN/result"
img_path = "/home1/gzy/NucleiSegmentation/High_CCRCC/Test/Images"
save_dir = "/home1/gzy/NucleiSegmentation/High_CCRCC/Test/Overlays"
ann_dir = "/home1/gzy/NucleiSegmentation/High_CCRCC/Test/Labels"
file_list = glob.glob('%s/*%s' % (img_path, '.png'))
file_list.sort() # ensure same order

In [9]:
img_path = "/home5/hby/PRCC/New_Data/dataset/2/TCGA-P4-A5E8-01Z-00-DX1.71EDB6C0-3FA1-4380-BFDB-C1B757E83C3A"
ann_dir = "/home5/gzy/PRCCDataset/Nuclei_Prediction_2000_new/TCGA-P4-A5E8-01Z-00-DX1.71EDB6C0-3FA1-4380-BFDB-C1B757E83C3A"
file_list = glob.glob('%s/*%s' % (img_path, '.png'))
file_list.sort() # ensure same order

In [15]:
for filename in tqdm(file_list):
    filename = os.path.basename(filename)
    basename = os.path.splitext(filename)[0]
    
    img = cv2.imread(img_path + '/' + filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
#     ann_inst, ann_type = load_ground_trues_ccrcc(ann_dir, basename)
    labels = sio.loadmat(ann_dir + '/' + basename + '.mat')
    ann_inst = labels['Instance']
    ann_type = labels['Type']
    
    if np.unique(ann_inst)[0] < 0:
        print(filename)
        continue
    if (np.unique(ann_inst).size-1) != ann_inst.max():
        ann_inst = remap_label(ann_inst)
    overlaid_output = visualize_instances(img, ann_inst, ann_type)
    overlaid_output = cv2.cvtColor(overlaid_output, cv2.COLOR_BGR2RGB)
    
    cv2.imwrite('%s/%s.png' % ('./', basename), overlaid_output)
    #cv2.imwrite('%s/%s.png' % (save_dir, basename), overlaid_output)

100%|██████████| 5/5 [00:47<00:00,  9.45s/it]


In [48]:
from metrics.stats_utils import get_fast_aji, get_dice_1, get_dice_2, get_fast_dice_2
bpq_all = []
mpq_all = []
dq_all =[]
sq_all = []
aji_all = []
dice_1_all = []
dice_2_all = []
type_names = ['Grade1','Grade2','Grade3','Endo']
total = [0 for i in range(inferer.nr_types-1)]
correct = [0 for i in range(inferer.nr_types-1)]
predict = [0 for i in range(inferer.nr_types-1)]

for filename in tqdm(file_list):
    filename = os.path.basename(filename)
    basename = os.path.splitext(filename)[0]
#     if 'prcc' in basename:
#         continue
    
    img = cv2.imread(img_path + '/' + filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    ann_inst, ann_type = load_ground_trues_ccrcc(ann_dir, basename)
    if np.unique(ann_inst)[0] < 0:
        continue
    if (np.unique(ann_inst).size-1) != ann_inst.max():
        ann_inst = remap_label(ann_inst)
    # load hover results
    pred = sio.loadmat(pred_path + '/' + basename + '.mat')
#     pred = pred['result']
#     pred_inst, pred_type = process_instance(pred, nr_types=inferer.nr_types)
#     pred_inst = np.load(inst_path + '/' + basename + '.npy')
#     pred_inst, pred_type = process_instance_hcnet(pred, nr_types=inferer.nr_types)
    pred_inst, pred_type = process_instance_mrcnn(pred)

#     # load dg results
#     pred = np.load(pred_path + '/' + basename + '.npy')
#     pred[pred==1] = 0
#     pred[pred==-1] = 0
#     pred = remap_label(pred)
#     pred_inst = pred
#     pred_type = pred.copy()
#     pred_type[pred_type!=0] = 1

    if (np.unique(pred_inst).size-1) != pred_inst.max():
        pred_inst = remap_label(pred_inst)

#     overlaid_output = visualize_instances(img, pred_inst, pred_type)
#     overlaid_output = cv2.cvtColor(overlaid_output, cv2.COLOR_BGR2RGB)
#     cv2.imwrite('%s/%s.png' % (save_dir, basename), overlaid_output)
#     continue

    a_dice_1 = get_dice_1(ann_inst, pred_inst)
    #a_dice_2 = get_fast_dice_2(ann_inst, pred_inst)
    a_ajis = get_fast_aji(ann_inst.astype('int'), pred_inst.astype('int'))

    dq, sq, bpq, mpq = cal_metrics(ann_inst, ann_type, pred_inst, pred_type)
    bpq_all.append(bpq)
    mpq_all.append(mpq)
    dq_all.append(dq)
    sq_all.append(sq)
    aji_all.append(a_ajis)
    dice_1_all.append(a_dice_1)
    #dice_2_all.append(a_dice_2)

grade1_PQ = np.nanmean([pq[0] for pq in mpq_all])
grade2_PQ = np.nanmean([pq[1] for pq in mpq_all])
grade3_PQ = np.nanmean([pq[2] for pq in mpq_all])
endothelium_PQ = np.nanmean([pq[3] for pq in mpq_all])
mpq_each_image = [np.nanmean(pq) for pq in mpq_all]

recall_pc = [correct[i]/total[i] for i in range(len(correct))]
precision_pc = [correct[i]/predict[i] for i in range(len(correct))]

recall = np.sum(correct)/np.sum(total)
precision = np.sum(correct)/np.sum(predict)
F1 = 2*recall*precision/(precision+recall)

print("The AJI is %.4f, The Dice1 is %.4f" % (np.nanmean(aji_all), np.nanmean(dice_1_all)))

print("The DQ is %.4f, The SQ is %.4f" % (np.nanmean(dq_all), np.nanmean(sq_all)))

print("The binary PQ is %.4f, The mean PQ is %.4f,\nThe PQ of %s is %.4f,\
        \nThe PQ of %s is %.4f, \nThe PQ of %s is %.4f, \nThe PQ of %s is %.4f"
      % (np.nanmean(bpq_all), np.nanmean(mpq_each_image), type_names[0], np.nanmean(grade1_PQ),
         type_names[1], np.nanmean(grade2_PQ), type_names[2], np.nanmean(grade3_PQ),
         type_names[3], np.nanmean(endothelium_PQ)))

print("Precision: %.4f, Recall: %.4f, F1: %.4f" % (precision, recall, F1))
print("            Pricision    Recall    F1")
print("%s:   %.4f      %.4f    %.4f" % (type_names[0], precision_pc[0], recall_pc[0], 2*recall_pc[0]*precision_pc[0]/(precision_pc[0]+recall_pc[0])))
print("%s:   %.4f      %.4f    %.4f" % (type_names[1], precision_pc[1], recall_pc[1], 2*recall_pc[1]*precision_pc[1]/(precision_pc[1]+recall_pc[1])))
print("%s:   %.4f      %.4f    %.4f" % (type_names[2], precision_pc[2], recall_pc[2], 2*recall_pc[2]*precision_pc[2]/(precision_pc[2]+recall_pc[2])))
print("%s:   %.4f      %.4f    %.4f" % (type_names[3], precision_pc[3], recall_pc[3], 2*recall_pc[3]*precision_pc[3]/(precision_pc[3]+recall_pc[3])))

100%|██████████| 200/200 [02:33<00:00,  1.30it/s]

The AJI is 0.6794, The Dice1 is 0.8257
The DQ is 0.8432, The SQ is 0.8212
The binary PQ is 0.6926, The mean PQ is 0.4393,
The PQ of Grade1 is 0.6297,        
The PQ of Grade2 is 0.2338, 
The PQ of Grade3 is 0.4423, 
The PQ of Endo is 0.3646
Precision: 0.7180, Recall: 0.6678, F1: 0.6920
            Pricision    Recall    F1
Grade1:   0.7771      0.8382    0.8065
Grade2:   0.4036      0.2608    0.3168
Grade3:   0.4778      0.4327    0.4541
Endo:   0.6565      0.4339    0.5225


In [11]:
# 生成预测overlap图
for filename in tqdm(file_list):
    filename = os.path.basename(filename)
    basename = os.path.splitext(filename)[0]
    img = cv2.imread(img_path + '/' + filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    pred = sio.loadmat(pred_path + '/' + basename + '.mat')
#     pred = pred['result']
#     ann_inst, ann_type = load_ground_trues_ccrcc(ann_dir, basename)

    pred_inst, pred_type = process_instance_mrcnn(pred)
#     pred_inst, pred_type = process_instance(pred, nr_types=inferer.nr_types)

    overlaid_output = visualize_instances(img, pred_inst, pred_type)
    overlaid_output = cv2.cvtColor(overlaid_output, cv2.COLOR_BGR2RGB)

    cv2.imwrite('%s/%s_maskrcnn.png' % (save_dir, basename), overlaid_output)

100%|██████████| 200/200 [00:37<00:00,  5.34it/s]


In [6]:
# 生成预测高斯结果图
for filename in tqdm(file_list):
    filename = os.path.basename(filename)
    basename = os.path.splitext(filename)[0]
    pred = sio.loadmat(pred_path + '/' + basename + '.mat')
    pred = pred['result'][0]
    pos_map = pred[...,-1]*255
#     bnd_map = np.argmax(bnd_map, axis=-1)
#     bnd_map = bnd_map*127.5

    cv2.imwrite('%s/%s.png' % (save_dir+'/pos_map/', basename), pos_map)
#     cv2.imwrite('%s/%s.png' % (save_dir+'/distmap_mae/', basename), n_map*255)

100%|██████████| 200/200 [00:01<00:00, 179.35it/s]
